# 4 - Modelos MA

## 4.1 Descripción

El siguiente modelo a evaluar es el de media movil (MA), definido como:

* Modelo de media móvil de orden $n_c$, MA($n_c$):

\begin{equation}
y(t) = C + c_1 \epsilon(t-1) + \dots + c_{n_c}\epsilon(t - n_c) + \epsilon(t)
\end{equation}



## 4.2 Generamos el conjunto de datos
Cargamos las librerías y los datos

In [1]:
# Importamos las librerias necesarias para trabajar
from statsmodels.regression.linear_model import yule_walker
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import mean_squared_error
from numpy.fft import fft, fftfreq, fftshift
from datetime import time
from matplotlib import rc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mysql.connector
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
import copy
import sklearn

# Seteamos el estilo de los graficos
sns.set(style="whitegrid")

# Configuramos los graficos con latex
plt.rc('text', usetex=True)

# Funcion para la metrica de la trayectoria
def error_trayectoria(vector):
    '''
    Se calcula una norma de error como la diferencia entre la prediccion y la medicion
    real para un tiempo t. Se utilizara la norma euclidiana
    '''
    y_predict = vector['y+1':'y+6']
    y_real = vector['y']
    error = (y_predict - y_real) ** 2
    error = sum(error) / len(error)
    return np.sqrt(error)

def phi_X(R_X, gamma, Ts=5 * 60):
    arg_max = R_X.argmax()
    R_X_wind = R_X[arg_max - gamma: arg_max + gamma + 1]
    wind = np.hanning(len(R_X_wind))
    phi_X = fft(R_X_wind * wind)
    freq = fftfreq(len(phi_X), Ts)
    phi_X = pd.Series(phi_X, index=freq)
    phi_X = phi_X[freq > 0]
    return phi_X

In [2]:
# Abrimos la base de datos
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    password='7461143',
    database='datos_ordenados'
)

# Extraemos la informacion en un dataframe
df = pd.read_sql("SELECT * FROM cgm_ordenados", mydb)   # Cargamos todos los datos 
#df.drop('id', axis=1, inplace=True)                   # Eliminamos el indice
df.set_index('datetime', inplace=True)                # Definimos datetime como indice
df.sort_index(inplace=True)                           # Ordenamos en base a datetime
df.index.freq = pd.infer_freq(df.index)
# Mostramos los resultados
print('Tamano de la tabla: {} filas y {} columnas'.format(df.shape[0], df.shape[1]))
print('Tiempo del estudio:')
print(' - Inicio  : {}'.format(str(df.index[0])))
print(' - Final   : {}'.format(str(df.index[-1])))
print(' - Duración: {}'.format(str(df.index[-1] - df.index[0])))
df.head(3)

Tamano de la tabla: 1728 filas y 6 columnas
Tiempo del estudio:
 - Inicio  : 2020-01-24 17:00:00
 - Final   : 2020-01-30 16:55:00
 - Duración: 5 days 23:55:00


,sensor_glucose,sensor_calibration_bg,meal,basal_insulin,bolus_insulin,exercise
datetime,,,,,,
2020-01-24 17:00:00,NaN,125.0,NaN,NaN,NaN,NaN
2020-01-24 17:05:00,126.0,NaN,NaN,NaN,NaN,NaN
2020-01-24 17:10:00,128.0,NaN,NaN,NaN,NaN,NaN


Extraemos y procesamos las variables utilizadas para el modelo

In [3]:
# en este caso, solo es la variable de la glucosa
y = copy.copy(df['sensor_glucose'])
# Realizamos una interpolacion para eliminar los NaN
y.interpolate(inplace=True, limit_direction='both')
# Cambiamos el nombre de la variable a y (por simplicidad a futuro)
y.rename('y', inplace=True)
# Por comodidad, se trabajara con los datos como Dataframe y no como Serie
y = pd.DataFrame(y)
y.head(5)

,y
datetime,
2020-01-24 17:00:00,126.0
2020-01-24 17:05:00,126.0
2020-01-24 17:10:00,128.0
2020-01-24 17:15:00,146.0
2020-01-24 17:20:00,158.0


Dividimos los datos en el conjunto de entrenamiento y de testeo

In [4]:
# Parametros
fecha_limite = '2020-01-28 16:59:59'
y_train = copy.copy(y[:fecha_limite])
y_test = copy.copy(y[fecha_limite:])
print('- porcentaje de training: {:2.2f}%'.format(100*len(y_train)/len(y)))
print('- porcentaje de testing : {:2.2f}%'.format(100*len(y_test)/len(y)))

- porcentaje de training: 66.67%
- porcentaje de testing : 33.33%


## 4.3 Busqueda del mejor modelo en base al RMSE

Buscaremos el mejor modelo en base a una iteración para $n_c \in [0,60]$

In [113]:
from statsmodels.tsa.arima_model import ARIMA
n_c = 8
n_a = 2
d = 0
model = ARIMA(y_train, order=(n_a, d, n_c))
model_fit = model.fit()
ar_params = model_fit.params


aa = model_fit.predict(0, 45)
ar_params = model_fit.params

In [121]:
params = model_fit.params
n =max(n_a, n_c)
yy = y[:n+3]
yy = yy['y'].to_numpy()
y_na = yy[-n_a:]
y_na = np.flip(y_na)
y_nc = np.diff(yy[-n_c-1:])
y_nc = y_nc[-n_c:]
y_nc = np.flip(y_nc)
y_sort = np.concatenate([[1], y_na])
y_sort = np.concatenate([y_sort,y_nc])
#print(pd.Series(y_sort))
#y[:n+1]

y_pred = np.inner(y_sort[1:], params[1:])

#print(pd.Series(y_sort))
#print(params)
print(y[:n+1])
y_pred
#aa[:n+4]

                         y
datetime                  
2020-01-24 17:00:00  126.0
2020-01-24 17:05:00  126.0
2020-01-24 17:10:00  128.0
2020-01-24 17:15:00  146.0
2020-01-24 17:20:00  158.0
2020-01-24 17:25:00  171.0
2020-01-24 17:30:00  216.0
2020-01-24 17:35:00  228.0
2020-01-24 17:40:00  246.0


275.83770099561866

In [5]:
from statsmodels.tsa.arima_model import ARIMA
# Variable que almacena los resultados
resultados = pd.DataFrame(columns=['train_1_paso','train_6_paso','train_trajec', 'test_1_paso','test_6_paso','test_trajec'])
resultados.index.name='n_c'
iterar = False      # Variable que activa la busqueda real. Si está en False, carga una que se realizó en una iteracion previa
n_c = 3
# Predicion de 6 pasos
k = 6

######################################
# Entrenamiento y generacion del modelo
######################################

# Entrenamos el modelo
# Modelos MA utiliza d=0 y n_a=0
n_a = 0
d = 0
model = ARIMA(y_train, order=(n_a, d, n_c))
model_fit = model.fit()
ar_params = model_fit.params

# Generamos el modelo
def ar_model(x):
    '''
    Modelo AR. Utiliza la variable globar de parametros
    '''
    global ar_params, n_a
    x = x[-n_a:]
    x = np.flip(x.append(pd.Series([1])))
    return np.inner(x, ar_params)

######################################
# Dataframe para predecir
######################################
Y_train = copy.copy(y_train)
Y_test = copy.copy(y_test)
for i in range(n_a):
    Y_train['y-{}'.format(i+1)] = y_train['y'].shift(i+1)
    Y_test['y-{}'.format(i+1)] = y_test['y'].shift(i+1)
Y_train = Y_train[Y_train.columns[::-1]]
Y_test = Y_test[Y_test.columns[::-1]]

######################################
# Prediccion
######################################
# Realizamos la prediccion
for i in range(k):
    Y_train['y+{}'.format(i+1)] = Y_train.apply(ar_model, axis=1)
    Y_test['y+{}'.format(i+1)] = Y_test.apply(ar_model, axis=1)

# Se debe desfasar la prediccion para que coincida con el valor el tiempo
for i in range(k):
    Y_train['y+{}'.format(i+1)] = Y_train['y+{}'.format(i+1)].shift(i+1)
    Y_test['y+{}'.format(i+1)] = Y_test['y+{}'.format(i+1)].shift(i+1)

######################################
# Calculo de los errores
######################################

# 1 paso adelante (5 minutos)
Y_train['e_1_paso'] = Y_train['y+1'] - Y_train['y']
Y_test['e_1_paso'] = Y_test['y+1'] - Y_test['y']

# 6 pasos adelante (30 minutos)
Y_train['e_6_paso'] = Y_train['y+6'] - Y_train['y']
Y_test['e_6_paso'] = Y_test['y+6'] - Y_test['y']

# Trayectoria (5 a 30 minutos)
Y_train['e_trajec'] = Y_train.apply(error_trayectoria, axis=1) 
Y_test['e_trajec'] = Y_test.apply(error_trayectoria, axis=1)

# Extraemos los vectores de los errores
e1 = copy.copy(Y_train['e_1_paso'])
e6 = copy.copy(Y_train['e_6_paso'])
ee = copy.copy(Y_train['e_trajec'])
# Eliminamos los puntos los nan
e1.dropna(inplace=True)
e6.dropna(inplace=True)
ee.dropna(inplace=True)
# Calculamos el resultado
train_RMSE_1_paso = np.sqrt(sum(e1 ** 2) / len(e1))
train_RMSE_6_paso = np.sqrt(sum(e6 ** 2) / len(e6))
train_RMSE_trajec = np.sqrt(sum(ee ** 2) / len(ee))
# Extraemos los vectores de los errores
e1 = copy.copy(Y_test['e_1_paso'])
e6 = copy.copy(Y_test['e_6_paso'])
ee = copy.copy(Y_test['e_trajec'])
# Eliminamos los puntos los NaN
e1.dropna(inplace=True)
e6.dropna(inplace=True)
ee.dropna(inplace=True)
# Calculamos el resultado
test_RMSE_1_paso = np.sqrt(sum(e1 ** 2) / len(e1))
test_RMSE_6_paso = np.sqrt(sum(e6 ** 2) / len(e6))
test_RMSE_trajec = np.sqrt(sum(ee ** 2) / len(ee))
# Agregamos el resultado a un dataframe
resultados.loc[n_a, 'train_1_paso'] = train_RMSE_1_paso
resultados.loc[n_a, 'train_6_paso'] = train_RMSE_6_paso
resultados.loc[n_a, 'train_trajec'] = train_RMSE_trajec

resultados.loc[n_a, 'test_1_paso'] = test_RMSE_1_paso
resultados.loc[n_a, 'test_6_paso'] = test_RMSE_6_paso
resultados.loc[n_a, 'test_trajec'] = test_RMSE_trajec

# Lo desplegamos por consola los resultados obtenidos
print(' - Training / testing para n_a: {}'.format(n_a))
print('RMSE 1 paso : {:.3f} / {:.3f}'.format(train_RMSE_1_paso, test_RMSE_1_paso))
print('RMSE 6 pasos: {:.3f} / {:.3f}'.format(train_RMSE_6_paso, test_RMSE_6_paso))
print('RMSE traject: {:.3f} / {:.3f}'.format(train_RMSE_trajec, test_RMSE_trajec))

# Guardamos los nuevos valores
resultados.to_csv('resultados.csv', sep=';')

C:\Users\Particular\AppData\Local\Programs\Python\Python36\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


ValueError: The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.

In [ ]:
from statsmodels.tsa.ar_model import AutoReg
# Variable que almacena los resultados
resultados = pd.DataFrame(columns=['train_1_paso','train_6_paso','train_trajec', 'test_1_paso','test_6_paso','test_trajec'])
resultados.index.name='n_a'
iterar = False      # Variable que activa la busqueda real. Si está en False, carga una que se realizó en una iteracion previa
n_a_max= 60
# Predicion de 6 pasos
k = 6

for n_a in range(1, n_a_max + 1):
    if iterar:
        
        ######################################
        # Entrenamiento y generacion del modelo
        ######################################
        
        # Entrenamos el modelo
        model = AutoReg(y_train, lags=n_a, old_names=False)
        model_fit = model.fit()
        ar_params = model_fit.params

        # Generamos el modelo
        def ar_model(x):
            '''
            Modelo AR. Utiliza la variable globar de parametros
            '''
            global ar_params, n_a
            x = x[-n_a:]
            x = np.flip(x.append(pd.Series([1])))
            return np.inner(x, ar_params)
        
        ######################################
        # Dataframe para predecir
        ######################################
        Y_train = copy.copy(y_train)
        Y_test = copy.copy(y_test)
        for i in range(n_a):
            Y_train['y-{}'.format(i+1)] = y_train['y'].shift(i+1)
            Y_test['y-{}'.format(i+1)] = y_test['y'].shift(i+1)
        Y_train = Y_train[Y_train.columns[::-1]]
        Y_test = Y_test[Y_test.columns[::-1]]

        ######################################
        # Prediccion
        ######################################
        # Realizamos la prediccion
        for i in range(k):
            Y_train['y+{}'.format(i+1)] = Y_train.apply(ar_model, axis=1)
            Y_test['y+{}'.format(i+1)] = Y_test.apply(ar_model, axis=1)

        # Se debe desfasar la prediccion para que coincida con el valor el tiempo
        for i in range(k):
            Y_train['y+{}'.format(i+1)] = Y_train['y+{}'.format(i+1)].shift(i+1)
            Y_test['y+{}'.format(i+1)] = Y_test['y+{}'.format(i+1)].shift(i+1)
        
        ######################################
        # Calculo de los errores
        ######################################
        
        # 1 paso adelante (5 minutos)
        Y_train['e_1_paso'] = Y_train['y+1'] - Y_train['y']
        Y_test['e_1_paso'] = Y_test['y+1'] - Y_test['y']

        # 6 pasos adelante (30 minutos)
        Y_train['e_6_paso'] = Y_train['y+6'] - Y_train['y']
        Y_test['e_6_paso'] = Y_test['y+6'] - Y_test['y']

        # Trayectoria (5 a 30 minutos)
        Y_train['e_trajec'] = Y_train.apply(error_trayectoria, axis=1) 
        Y_test['e_trajec'] = Y_test.apply(error_trayectoria, axis=1)
        
        # Extraemos los vectores de los errores
        e1 = copy.copy(Y_train['e_1_paso'])
        e6 = copy.copy(Y_train['e_6_paso'])
        ee = copy.copy(Y_train['e_trajec'])
        # Eliminamos los puntos los nan
        e1.dropna(inplace=True)
        e6.dropna(inplace=True)
        ee.dropna(inplace=True)
        # Calculamos el resultado
        train_RMSE_1_paso = np.sqrt(sum(e1 ** 2) / len(e1))
        train_RMSE_6_paso = np.sqrt(sum(e6 ** 2) / len(e6))
        train_RMSE_trajec = np.sqrt(sum(ee ** 2) / len(ee))
        # Extraemos los vectores de los errores
        e1 = copy.copy(Y_test['e_1_paso'])
        e6 = copy.copy(Y_test['e_6_paso'])
        ee = copy.copy(Y_test['e_trajec'])
        # Eliminamos los puntos los NaN
        e1.dropna(inplace=True)
        e6.dropna(inplace=True)
        ee.dropna(inplace=True)
        # Calculamos el resultado
        test_RMSE_1_paso = np.sqrt(sum(e1 ** 2) / len(e1))
        test_RMSE_6_paso = np.sqrt(sum(e6 ** 2) / len(e6))
        test_RMSE_trajec = np.sqrt(sum(ee ** 2) / len(ee))
        # Agregamos el resultado a un dataframe
        resultados.loc[n_a, 'train_1_paso'] = train_RMSE_1_paso
        resultados.loc[n_a, 'train_6_paso'] = train_RMSE_6_paso
        resultados.loc[n_a, 'train_trajec'] = train_RMSE_trajec

        resultados.loc[n_a, 'test_1_paso'] = test_RMSE_1_paso
        resultados.loc[n_a, 'test_6_paso'] = test_RMSE_6_paso
        resultados.loc[n_a, 'test_trajec'] = test_RMSE_trajec
        
        # Lo desplegamos por consola los resultados obtenidos
        print(' - Training / testing para n_a: {}'.format(n_a))
        print('RMSE 1 paso : {:.3f} / {:.3f}'.format(train_RMSE_1_paso, test_RMSE_1_paso))
        print('RMSE 6 pasos: {:.3f} / {:.3f}'.format(train_RMSE_6_paso, test_RMSE_6_paso))
        print('RMSE traject: {:.3f} / {:.3f}'.format(train_RMSE_trajec, test_RMSE_trajec))
        
        # Guardamos los nuevos valores
        resultados.to_csv('resultados.csv', sep=';')
    else:
        resultados = pd.read_csv('resultados.csv', sep=';')
        resultados.set_index('n_a', inplace=True)

In [ ]:
resultados.tail(3)

In [ ]:
# Graficamos
fig, ax = plt.subplots()

ax.plot(resultados['train_1_paso'], color='C0', linestyle='--', label='train RMSE$_{t + 1}$', marker='')
ax.plot(resultados['test_1_paso'], color='C0', linestyle=':', label='test RMSE$_{t + 1}$', marker='')

ax.plot(resultados['train_6_paso'], color='C1', linestyle='--', label='train RMSE$_{t + 6}$')
ax.plot(resultados['test_6_paso'], color='C1', linestyle=':', label='test RMSE$_{t + 6}$')

ax.plot(resultados['train_trajec'], color='C2', linestyle='--', label='train RMSE$_{trajectory}$')
ax.plot(resultados['test_trajec'], color='C2', linestyle=':', label='test RMSE$_{trajectory}$')

# Parametros
ax.set_xticks(resultados.index, minor=True)
ax.set_xticks(np.arange(0, max(resultados.index)+0.5, 5))
ax.grid(b=True, which='major', color='k', alpha=0.2)
ax.grid(b=True, which='minor', color='k', alpha=0.2, linestyle=':')
ax.set_xlim([0, max(resultados.index)])
ax.set_ylabel('Glucose [mg/dL]')
ax.set_xlabel('Order $n_a$')
ax.legend(fancybox=True, shadow=True, ncol=3)

y_size = 4.2
x_size = 3 * y_size
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/error_grafico_1'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

In [ ]:
print('n_a que minimiza el conjunto de entrenamiento para 1 paso:      {}'.format(resultados['train_1_paso'].argmin()))
print('n_a que minimiza el conjunto de entrenamiento para 6 paso:      {}'.format(resultados['train_6_paso'].argmin()))
print('n_a que minimiza el conjunto de entrenamiento para tarjectoria: {}'.format(resultados['train_trajec'].argmin()))
print('n_a que minimiza el conjunto de prueba para 1 paso:      {}'.format(resultados['test_1_paso'].argmin()))
print('n_a que minimiza el conjunto de prueba para 6 paso:      {}'.format(resultados['test_6_paso'].argmin()))
print('n_a que minimiza el conjunto de prueba para tarjectoria: {}'.format(resultados['test_trajec'].argmin()))

obtenemos los resultados para los ordenes $n_a$ relevantes. Estos serán:
* Según el paper, se probaron $n_a =3, 6, 9 y 12$
* Visualmente vemos que para $n_a = 2$ el error tiene un cambio importante y es un modelo simple
* Adicionalmente, la función de autocorrelación parcial para CGM diferencia indica $n_a = 2$
* El error que minimiza el training $n_a=59$
* El error que minimiza el testing $n_a=25,30$

In [ ]:
n_a_relevantes = [2, 3, 6, 9, 12, 25, 30, 59]
resultados.loc[n_a_relevantes,:]

Se verá el desempeño para $n_a=2$, dado que es el resultado más simple con buen desempeño (y el orden de PACF) y para $n_a=30$, dado que es uno de los que minimiza el testing

## 2.4 Resultados para $n_a=2$

### Entrenamos el modelo

In [ ]:
######################################
# Entrenamiento y generacion del modelo
######################################
n_a = 2




# Entrenamos el modelo
model = AutoReg(y_train, lags=n_a, old_names=False)
model_fit = model.fit()
ar_params = model_fit.params

# Generamos el modelo
def ar_model(x):
    '''
    Modelo AR. Utiliza la variable globar de parametros
    '''
    global ar_params, n_a
    x = x[-n_a:]
    x = np.flip(x.append(pd.Series([1])))
    return np.inner(x, ar_params)

######################################
# Dataframe para predecir
######################################
Y_train = copy.copy(y_train)
Y_test = copy.copy(y_test)
for i in range(n_a):
    Y_train['y-{}'.format(i+1)] = y_train['y'].shift(i+1)
    Y_test['y-{}'.format(i+1)] = y_test['y'].shift(i+1)
Y_train = Y_train[Y_train.columns[::-1]]
Y_test = Y_test[Y_test.columns[::-1]]

######################################
# Prediccion
######################################
# Realizamos la prediccion
for i in range(k):
    Y_train['y+{}'.format(i+1)] = Y_train.apply(ar_model, axis=1)
    Y_test['y+{}'.format(i+1)] = Y_test.apply(ar_model, axis=1)

# Se debe desfasar la prediccion para que coincida con el valor el tiempo
for i in range(k):
    Y_train['y+{}'.format(i+1)] = Y_train['y+{}'.format(i+1)].shift(i+1)
    Y_test['y+{}'.format(i+1)] = Y_test['y+{}'.format(i+1)].shift(i+1)

######################################
# Calculo de los errores
######################################

# 1 paso adelante (5 minutos)
Y_train['e_1_paso'] = Y_train['y+1'] - Y_train['y']
Y_test['e_1_paso'] = Y_test['y+1'] - Y_test['y']

# 6 pasos adelante (30 minutos)
Y_train['e_6_paso'] = Y_train['y+6'] - Y_train['y']
Y_test['e_6_paso'] = Y_test['y+6'] - Y_test['y']

# Trayectoria (5 a 30 minutos)
Y_train['e_trajec'] = Y_train.apply(error_trayectoria, axis=1) 
Y_test['e_trajec'] = Y_test.apply(error_trayectoria, axis=1)

# Extraemos los vectores de los errores
e1 = copy.copy(Y_train['e_1_paso'])
e6 = copy.copy(Y_train['e_6_paso'])
ee = copy.copy(Y_train['e_trajec'])
# Eliminamos los puntos los nan
e1.dropna(inplace=True)
e6.dropna(inplace=True)
ee.dropna(inplace=True)
# Calculamos el resultado
train_RMSE_1_paso = np.sqrt(sum(e1 ** 2) / len(e1))
train_RMSE_6_paso = np.sqrt(sum(e6 ** 2) / len(e6))
train_RMSE_trajec = np.sqrt(sum(ee ** 2) / len(ee))
# Extraemos los vectores de los errores
e1 = copy.copy(Y_test['e_1_paso'])
e6 = copy.copy(Y_test['e_6_paso'])
ee = copy.copy(Y_test['e_trajec'])
# Eliminamos los puntos los NaN
e1.dropna(inplace=True)
e6.dropna(inplace=True)
ee.dropna(inplace=True)
# Calculamos el resultado
test_RMSE_1_paso = np.sqrt(sum(e1 ** 2) / len(e1))
test_RMSE_6_paso = np.sqrt(sum(e6 ** 2) / len(e6))
test_RMSE_trajec = np.sqrt(sum(ee ** 2) / len(ee))

Resumen del modelo

In [ ]:
model_fit.summary()

### Análisis estadístico del error

In [ ]:
Y_train[['e_1_paso', 'e_6_paso', 'e_trajec']].describe()

In [ ]:
Y_test[['e_1_paso', 'e_6_paso', 'e_trajec']].describe()

In [ ]:
# Creamos la figura
fig, (ax1, ax2) = plt.subplots(1, 2)

#Parametros del grafico
data1 = [Y_train['e_1_paso'].dropna(), Y_train['e_6_paso'].dropna(), Y_train['e_trajec'].dropna()]
data2 = [Y_test['e_1_paso'].dropna(), Y_test['e_6_paso'].dropna(), Y_test['e_trajec'].dropna()]
colors = ['C0', 'C1', 'C2']
n_bins = 45
labels = ['$e_{t+1}$', '$e_{t+6}$', '$e_{trajectory}$']

# Realizamos el histograma
ax1.hist(data1, color=colors, bins=n_bins, label=labels, stacked=True)
ax2.hist(data2, color=colors, bins=n_bins, label=labels, stacked=True)

# Configuraciones
ax1.grid(True)
ax2.grid(True)
ax1.set_ylabel('Count')
ax1.set_xlabel('Glucose [mg/dL]')
ax2.set_xlabel('Glucose [mg/dL]')
ax1.legend(fancybox=True, shadow=True)
ax2.legend(fancybox=True, shadow=True)
ax1.set_title('(a)', loc='left')
ax2.set_title('(b)', loc='left')


x_size = 10
y_size = x_size / 3
fig.set_size_inches(x_size, y_size)
plt.tight_layout()
format_name = 'figs/error_histograma_na_2'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

### Cálculo de indicadores de desempeño

In [ ]:
# Extraemos los vectores de los errores
e1 = copy.copy(Y_train['e_1_paso'])
e6 = copy.copy(Y_train['e_6_paso'])
ee = copy.copy(Y_train['e_trajec'])
# Eliminamos los puntos los nan
e1.dropna(inplace=True)
e6.dropna(inplace=True)
ee.dropna(inplace=True)
# Calculamos el resultado
RMSE_1_paso = np.sqrt(sum(e1 ** 2) / len(e1))
RMSE_6_paso = np.sqrt(sum(e6 ** 2) / len(e6))
RMSE_trajec = np.sqrt(sum(ee ** 2) / len(ee))
# Agregamos el resultado a un dataframe
resultados.loc['RMSE', '1 paso'] = RMSE_1_paso
resultados.loc['RMSE', '6 paso'] = RMSE_6_paso
resultados.loc['RMSE', 'trajec'] = RMSE_trajec
# Lo desplegamos por consola
print(' - Training')
print('RMSE 1 paso : {:.3f}'.format(RMSE_1_paso))
print('RMSE 6 pasos: {:.3f}'.format(RMSE_6_paso))
print('RMSE traject: {:.3f}'.format(RMSE_trajec))

In [ ]:
# Extraemos los vectores de los errores
e1 = copy.copy(Y_test['e_1_paso'])
e6 = copy.copy(Y_test['e_6_paso'])
ee = copy.copy(Y_test['e_trajec'])
# Eliminamos los puntos los nan
e1.dropna(inplace=True)
e6.dropna(inplace=True)
ee.dropna(inplace=True)
# Calculamos el resultado
RMSE_1_paso = np.sqrt(sum(e1 ** 2) / len(e1))
RMSE_6_paso = np.sqrt(sum(e6 ** 2) / len(e6))
RMSE_trajec = np.sqrt(sum(ee ** 2) / len(ee))
# Agregamos el resultado a un dataframe
resultados.loc['RMSE', '1 paso'] = RMSE_1_paso
resultados.loc['RMSE', '6 paso'] = RMSE_6_paso
resultados.loc['RMSE', 'trajec'] = RMSE_trajec
# Lo desplegamos por consola
print(' - Testing')
print('RMSE 1 paso : {:.3f}'.format(RMSE_1_paso))
print('RMSE 6 pasos: {:.3f}'.format(RMSE_6_paso))
print('RMSE traject: {:.3f}'.format(RMSE_trajec))

In [ ]:
# Parametros
f_ini = pd.Timestamp('2020-01-29 00:00:00')
f_fin = pd.Timestamp('2020-01-29 23:59:59')

# Graficamos
fig, ax = plt.subplots()

ax.plot(Y_test['y+6'][f_ini: f_fin], color='lightcoral', 
        linestyle='--', label='$\hat{y}_{t + 6}$')
ax.plot(Y_test['y+5'][f_ini: f_fin], color='indianred', 
        linestyle='--', label='$\hat{y}_{t + 5}$')
ax.plot(Y_test['y+4'][f_ini: f_fin], color='brown', 
        linestyle='--', label='$\hat{y}_{t + 4}$')
ax.plot(Y_test['y+3'][f_ini: f_fin], color='firebrick', 
        linestyle='--', label='$\hat{y}_{t + 3}$')
ax.plot(Y_test['y+2'][f_ini: f_fin], color='maroon', 
        linestyle='--', label='$\hat{y}_{t + 2}$')
ax.plot(Y_test['y+1'][f_ini: f_fin], color='darkred', 
        linestyle='--', label='$\hat{y}_{t + 1}$')
ax.plot(Y_test['y'][f_ini: f_fin], color='C0', linewidth=3.0
       , label='$y_{t}$')

# Parametros
ax.grid(True)
ax.set_ylim([0, 450])
ax.set_xlim([f_ini, f_fin])
ax.set_ylabel('Glucose [mg/dL]')
ax.set_xlabel('Time')

# leyenda
handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
#labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles, labels, loc= 'upper center', ncol=4, fancybox=True, shadow=True)

# Formato de las fechas
date_form = mdates.DateFormatter('%H:%M')
ax.xaxis.set_major_formatter(date_form)

y_size = 4.2
x_size = 3 * y_size
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/grafico_tiempo_na_2_1'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

In [ ]:
# Parametros
f_ini = pd.Timestamp('2020-01-29 00:00:00')
f_fin = pd.Timestamp('2020-01-29 23:59:59')

# Graficamos
fig, ax = plt.subplots()

ax.plot(Y_test['y+6'][f_ini: f_fin], color='lightcoral', 
        linestyle='--', label='$\hat{y}_{t + 6}$')
ax.plot(Y_test['y'][f_ini: f_fin], color='C0', linewidth=3.0
       , label='$y_{t}$')

# Parametros
ax.grid(True)
ax.set_ylim([0, 450])
ax.set_xlim([f_ini, f_fin])
ax.set_ylabel('Glucosa [mg/dL]')
ax.set_xlabel('Tiempo')

# leyenda
handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
#labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles, labels, loc= 'upper center', ncol=4, fancybox=True, shadow=True)

# Formato de las fechas
date_form = mdates.DateFormatter('%H:%M')
ax.xaxis.set_major_formatter(date_form)

y_size = 4.2
x_size = 3 * y_size
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/grafico_tiempo_na_2_2'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

### Gráfico del error

* Gráficamos el error del training

In [ ]:
# Parametros
f_ini = y_train.index[0]
f_fin = y_train.index[-1]

# Graficamos
fig, ax = plt.subplots()

ax.plot(Y_train['e_1_paso'][f_ini: f_fin], color='C0', linestyle='-', label='$|e_{t + 1}|$')
ax.plot(Y_train['e_6_paso'][f_ini: f_fin], color='C1', linestyle='-', label='$|e_{t + 6}|$')
ax.plot(Y_train['e_trajec'][f_ini: f_fin], color='C2', linestyle='-', label='$e_{trajectory}$')

# Parametros
ax.grid(True)
ax.set_ylim([-150, 150])
ax.set_xlim([f_ini, f_fin])
ax.set_ylabel('Glucose [mg/dL]')
ax.set_xlabel('Time')

# leyenda
ax.legend(ncol=4, fancybox=True, shadow=True)

# Formato de las fechas
date_form = mdates.DateFormatter('%d-%m %H:%M')
ax.xaxis.set_major_formatter(date_form)

y_size = 4.2
x_size = 3 * y_size
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/error_grafico_training_n_a_2'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

* Graficamos el error del testing

In [ ]:
# Parametros
f_ini = y_test.index[0]
f_fin = y_test.index[-1]

# Graficamos
fig, ax = plt.subplots()

ax.plot(Y_test['e_1_paso'][f_ini: f_fin], color='C0', linestyle='-', label='$|e_{t + 1}|$')
ax.plot(Y_test['e_6_paso'][f_ini: f_fin], color='C1', linestyle='-', label='$|e_{t + 6}|$')
ax.plot(Y_test['e_trajec'][f_ini: f_fin], color='C2', linestyle='-', label='$e_{trajectory}$')

# Parametros
ax.grid(True)
ax.set_ylim([-150, 150])
ax.set_xlim([f_ini, f_fin])
ax.set_ylabel('Glucose [mg/dL]')
ax.set_xlabel('Time')

# leyenda
ax.legend(ncol=4, fancybox=True, shadow=True)

# Formato de las fechas
date_form = mdates.DateFormatter('%d-%m %H:%M')
ax.xaxis.set_major_formatter(date_form)

y_size = 4.2
x_size = 3 * y_size
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/error_grafico_testing_na_2'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

### Análisis en frecuencia del error

* Periodograma para el training

In [ ]:
e1 = Y_train['e_1_paso'].dropna()
N = len(e1)
freq = fftfreq(N, 5*60)
E1 = fft(e1, norm='ortho')
E1_N = abs(E1) ** 2
E1_N = pd.Series(E1_N, index=freq)
E1_N = E1_N[freq > 0]

e6 = Y_train['e_6_paso'].dropna()
N = len(e6)
freq = fftfreq(N, 5*60)
E6 = fft(e6, norm='ortho')
E6_N = abs(E6) ** 2
E6_N = pd.Series(E6_N, index=freq)
E6_N = E6_N[freq > 0]

ee = Y_train['e_trajec'].dropna()
N = len(ee)
freq = fftfreq(N, 5*60)
EE = fft(ee, norm='ortho')
EE_N = abs(EE) ** 2
EE_N = pd.Series(EE_N, index=freq)
EE_N = EE_N[freq > 0]

In [ ]:
# Creamos la figura y el axis
fig, (ax1, ax2) = plt.subplots(1, 2)

# Realizamos el grafico
ax1.loglog(E1_N, color='C0', label=r'$e_{t+1}$')
ax1.loglog(E6_N, color='C1', label=r'$e_{t+6}$')
ax1.loglog(EE_N, color='C2', label=r'$e_{trajectory}$')

ax2.semilogy(E1_N, color='C0', label=r'$e_{t+1}$')
ax2.semilogy(E6_N, color='C1', label=r'$e_{t+6}$')
ax2.semilogy(EE_N, color='C2', label=r'$e_{trajectory}$')

# Configuramos los parametros
ax1.grid(True, which='both')
ax1.set_ylim([10 ** (-2), 10 ** 5])
ax1.legend(fancybox=True, shadow=True)

ax2.grid(True, which='both')
ax2.set_ylim([10 ** (-2), 10 ** 5])
ax2.legend(fancybox=True, shadow=True)

ax1.set_ylabel(r'$\frac{Glucose^2}{Hz}$ [mg$^2$/dL$^2$/Hz]')
ax1.set_xlabel(r'Frequency [Hz]')
ax2.set_xlabel(r'Frequency [Hz]')

x_size = 3 * 4.2
y_size = 1 * x_size / 3
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/error_periodograma_training_na_2'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

* Periodograma para el testing

In [ ]:
e1 = Y_test['e_1_paso'].dropna()
N = len(e1)
freq = fftfreq(N, 5*60)
E1 = fft(e1, norm='ortho')
E1_N = abs(E1) ** 2
E1_N = pd.Series(E1_N, index=freq)
E1_N = E1_N[freq > 0]

e6 = Y_test['e_6_paso'].dropna()
N = len(e6)
freq = fftfreq(N, 5*60)
E6 = fft(e6, norm='ortho')
E6_N = abs(E6) ** 2
E6_N = pd.Series(E6_N, index=freq)
E6_N = E6_N[freq > 0]

ee = Y_test['e_trajec'].dropna()
N = len(ee)
freq = fftfreq(N, 5*60)
EE = fft(ee, norm='ortho')
EE_N = abs(EE) ** 2
EE_N = pd.Series(EE_N, index=freq)
EE_N = EE_N[freq > 0]

In [ ]:
# Creamos la figura y el axis
fig, (ax1, ax2) = plt.subplots(1, 2)

# Realizamos el grafico
ax1.loglog(E1_N, color='C0', label=r'$e_{t+1}$')
ax1.loglog(E6_N, color='C1', label=r'$e_{t+6}$')
ax1.loglog(EE_N, color='C2', label=r'$e_{trajectory}$')

ax2.semilogy(E1_N, color='C0', label=r'$e_{t+1}$')
ax2.semilogy(E6_N, color='C1', label=r'$e_{t+6}$')
ax2.semilogy(EE_N, color='C2', label=r'$e_{trajectory}$')

# Configuramos los parametros
ax1.grid(True, which='both')
ax1.set_ylim([10 ** (-2), 10 ** 5])
ax1.legend(fancybox=True, shadow=True)

ax2.grid(True, which='both')
ax2.set_ylim([10 ** (-2), 10 ** 5])
ax2.legend(fancybox=True, shadow=True)

ax1.set_ylabel(r'$\frac{Glucose^2}{Hz}$ [mg$^2$/dL$^2$/Hz]')
ax1.set_xlabel(r'Frequency [Hz]')
ax2.set_xlabel(r'Frequency [Hz]')

x_size = 3 * 4.2
y_size = 1 * x_size / 3
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/error_periodograma_testing_na_2'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

* Espectro para el training

In [ ]:
e1 = Y_train['e_1_paso'].dropna()
e6 = Y_train['e_6_paso'].dropna()
ee = Y_train['e_trajec'].dropna()

R_e1 = np.correlate(e1, e1, mode='full') / N
R_e6 = np.correlate(e6, e6, mode='full') / N
R_ee = np.correlate(ee, ee, mode='full') / N

N1 = len(R_e1)
N6 = len(R_e6)
Ne = len(R_ee)

# Gamma = N/2
gamma = round(N1 / 2) - 1
phi_E1 = phi_X(R_e1, gamma)

gamma = round(N6 / 2) - 1
phi_E6 = phi_X(R_e6, gamma)

gamma = round(Ne / 2) - 1
phi_EE = phi_X(R_ee, gamma)

In [ ]:
# Creamos la figura y el axis
fig, (ax1, ax2) = plt.subplots(1, 2)

# Realizamos el grafico
ax1.loglog(abs(phi_E1), color='C0', label=r'$e_{t+1}$')
ax1.loglog(abs(phi_E6), color='C1', label=r'$e_{t+6}$')
ax1.loglog(abs(phi_EE), color='C2', label=r'$e_{trajectory}$')

ax2.semilogy(abs(phi_E1), color='C0', label=r'$e_{t+1}$')
ax2.semilogy(abs(phi_E6), color='C1', label=r'$e_{t+6}$')
ax2.semilogy(abs(phi_EE), color='C2', label=r'$e_{trajectory}$')

# Configuramos los parametros
ax1.grid(True, which='both')
#ax1.set_ylim([10 ** (-2), 10 ** 6])
#ax1.set_xlim([10 ** (-6), max(Y_N.index)])
ax1.legend(fancybox=True, shadow=True)

ax2.grid(True, which='both')
#ax2.set_ylim([10 ** (-2), 10 ** 6])
#ax2.set_xlim([10 ** (-6), max(Y_N.index)])
ax2.legend(fancybox=True, shadow=True)

ax1.set_ylabel(r'$\frac{Glucose^2}{Hz}$ [mg$^2$/dL$^2$/Hz]')
ax1.set_xlabel(r'Frequency [Hz]')
ax2.set_xlabel(r'Frequency [Hz]')

x_size = 3 * 4.2
y_size = 1 * x_size / 3
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/error_espectro_training_na_2'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

* Espectro para el testing

In [ ]:
e1 = Y_test['e_1_paso'].dropna()
e6 = Y_test['e_6_paso'].dropna()
ee = Y_test['e_trajec'].dropna()

R_e1 = np.correlate(e1, e1, mode='full') / N
R_e6 = np.correlate(e6, e6, mode='full') / N
R_ee = np.correlate(ee, ee, mode='full') / N

N1 = len(R_e1)
N6 = len(R_e6)
Ne = len(R_ee)

# Gamma = N/2
gamma = round(N1 / 2) - 1
phi_E1 = phi_X(R_e1, gamma)

gamma = round(N6 / 2) - 1
phi_E6 = phi_X(R_e6, gamma)

gamma = round(Ne / 2) - 1
phi_EE = phi_X(R_ee, gamma)

In [ ]:
# Creamos la figura y el axis
fig, (ax1, ax2) = plt.subplots(1, 2)

# Realizamos el grafico
ax1.loglog(abs(phi_E1), color='C0', label=r'$e_{t+1}$')
ax1.loglog(abs(phi_E6), color='C1', label=r'$e_{t+6}$')
ax1.loglog(abs(phi_EE), color='C2', label=r'$e_{trajectory}$')

ax2.semilogy(abs(phi_E1), color='C0', label=r'$e_{t+1}$')
ax2.semilogy(abs(phi_E6), color='C1', label=r'$e_{t+6}$')
ax2.semilogy(abs(phi_EE), color='C2', label=r'$e_{trajectory}$')

# Configuramos los parametros
ax1.grid(True, which='both')
#ax1.set_ylim([10 ** (-2), 10 ** 6])
#ax1.set_xlim([10 ** (-6), max(Y_N.index)])
ax1.legend(fancybox=True, shadow=True)

ax2.grid(True, which='both')
#ax2.set_ylim([10 ** (-2), 10 ** 6])
#ax2.set_xlim([10 ** (-6), max(Y_N.index)])
ax2.legend(fancybox=True, shadow=True)

ax1.set_ylabel(r'$\frac{Glucose^2}{Hz}$ [mg$^2$/dL$^2$/Hz]')
ax1.set_xlabel(r'Frequency [Hz]')
ax2.set_xlabel(r'Frequency [Hz]')

x_size = 3 * 4.2
y_size = 1 * x_size / 3
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/error_espectro_testing_na_2'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

## 2.5 Resultados para $n_a=30$

### Entrenamos el modelo

In [ ]:
######################################
# Entrenamiento y generacion del modelo
######################################
n_a = 30

# Entrenamos el modelo
model = AutoReg(y_train, lags=n_a, old_names=False)
model_fit = model.fit()
ar_params = model_fit.params

# Generamos el modelo
def ar_model(x):
    '''
    Modelo AR. Utiliza la variable globar de parametros
    '''
    global ar_params, n_a
    x = x[-n_a:]
    x = np.flip(x.append(pd.Series([1])))
    return np.inner(x, ar_params)

######################################
# Dataframe para predecir
######################################
Y_train = copy.copy(y_train)
Y_test = copy.copy(y_test)
for i in range(n_a):
    Y_train['y-{}'.format(i+1)] = y_train['y'].shift(i+1)
    Y_test['y-{}'.format(i+1)] = y_test['y'].shift(i+1)
Y_train = Y_train[Y_train.columns[::-1]]
Y_test = Y_test[Y_test.columns[::-1]]

######################################
# Prediccion
######################################
# Realizamos la prediccion
for i in range(k):
    Y_train['y+{}'.format(i+1)] = Y_train.apply(ar_model, axis=1)
    Y_test['y+{}'.format(i+1)] = Y_test.apply(ar_model, axis=1)

# Se debe desfasar la prediccion para que coincida con el valor el tiempo
for i in range(k):
    Y_train['y+{}'.format(i+1)] = Y_train['y+{}'.format(i+1)].shift(i+1)
    Y_test['y+{}'.format(i+1)] = Y_test['y+{}'.format(i+1)].shift(i+1)

######################################
# Calculo de los errores
######################################

# 1 paso adelante (5 minutos)
Y_train['e_1_paso'] = Y_train['y+1'] - Y_train['y']
Y_test['e_1_paso'] = Y_test['y+1'] - Y_test['y']

# 6 pasos adelante (30 minutos)
Y_train['e_6_paso'] = Y_train['y+6'] - Y_train['y']
Y_test['e_6_paso'] = Y_test['y+6'] - Y_test['y']

# Trayectoria (5 a 30 minutos)
Y_train['e_trajec'] = Y_train.apply(error_trayectoria, axis=1) 
Y_test['e_trajec'] = Y_test.apply(error_trayectoria, axis=1)

# Extraemos los vectores de los errores
e1 = copy.copy(Y_train['e_1_paso'])
e6 = copy.copy(Y_train['e_6_paso'])
ee = copy.copy(Y_train['e_trajec'])
# Eliminamos los puntos los nan
e1.dropna(inplace=True)
e6.dropna(inplace=True)
ee.dropna(inplace=True)
# Calculamos el resultado
train_RMSE_1_paso = np.sqrt(sum(e1 ** 2) / len(e1))
train_RMSE_6_paso = np.sqrt(sum(e6 ** 2) / len(e6))
train_RMSE_trajec = np.sqrt(sum(ee ** 2) / len(ee))
# Extraemos los vectores de los errores
e1 = copy.copy(Y_test['e_1_paso'])
e6 = copy.copy(Y_test['e_6_paso'])
ee = copy.copy(Y_test['e_trajec'])
# Eliminamos los puntos los NaN
e1.dropna(inplace=True)
e6.dropna(inplace=True)
ee.dropna(inplace=True)
# Calculamos el resultado
test_RMSE_1_paso = np.sqrt(sum(e1 ** 2) / len(e1))
test_RMSE_6_paso = np.sqrt(sum(e6 ** 2) / len(e6))
test_RMSE_trajec = np.sqrt(sum(ee ** 2) / len(ee))

Resumen del modelo

In [ ]:
model_fit.summary()

### Análisis estadístico del error

In [ ]:
Y_train[['e_1_paso', 'e_6_paso', 'e_trajec']].describe()

In [ ]:
Y_test[['e_1_paso', 'e_6_paso', 'e_trajec']].describe()

In [ ]:
# Creamos la figura
fig, (ax1, ax2) = plt.subplots(1, 2)

#Parametros del grafico
data1 = [Y_train['e_1_paso'].dropna(), Y_train['e_6_paso'].dropna(), Y_train['e_trajec'].dropna()]
data2 = [Y_test['e_1_paso'].dropna(), Y_test['e_6_paso'].dropna(), Y_test['e_trajec'].dropna()]
colors = ['C0', 'C1', 'C2']
n_bins = 45
labels = ['$e_{t+1}$', '$e_{t+6}$', '$e_{trajectory}$']

# Realizamos el histograma
ax1.hist(data1, color=colors, bins=n_bins, label=labels, stacked=True)
ax2.hist(data2, color=colors, bins=n_bins, label=labels, stacked=True)

# Configuraciones
ax1.grid(True)
ax2.grid(True)
ax1.set_ylabel('Count')
ax1.set_xlabel('Glucose [mg/dL]')
ax2.set_xlabel('Glucose [mg/dL]')
ax1.legend(fancybox=True, shadow=True)
ax2.legend(fancybox=True, shadow=True)
ax1.set_title('(a)', loc='left')
ax2.set_title('(b)', loc='left')


x_size = 10
y_size = x_size / 3
fig.set_size_inches(x_size, y_size)
plt.tight_layout()
format_name = 'figs/error_histograma_na_30'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

### Cálculo de indicadores de desempeño

In [ ]:
# Extraemos los vectores de los errores
e1 = copy.copy(Y_train['e_1_paso'])
e6 = copy.copy(Y_train['e_6_paso'])
ee = copy.copy(Y_train['e_trajec'])
# Eliminamos los puntos los nan
e1.dropna(inplace=True)
e6.dropna(inplace=True)
ee.dropna(inplace=True)
# Calculamos el resultado
RMSE_1_paso = np.sqrt(sum(e1 ** 2) / len(e1))
RMSE_6_paso = np.sqrt(sum(e6 ** 2) / len(e6))
RMSE_trajec = np.sqrt(sum(ee ** 2) / len(ee))
# Agregamos el resultado a un dataframe
resultados.loc['RMSE', '1 paso'] = RMSE_1_paso
resultados.loc['RMSE', '6 paso'] = RMSE_6_paso
resultados.loc['RMSE', 'trajec'] = RMSE_trajec
# Lo desplegamos por consola
print(' - Training')
print('RMSE 1 paso : {:.3f}'.format(RMSE_1_paso))
print('RMSE 6 pasos: {:.3f}'.format(RMSE_6_paso))
print('RMSE traject: {:.3f}'.format(RMSE_trajec))

In [ ]:
# Extraemos los vectores de los errores
e1 = copy.copy(Y_test['e_1_paso'])
e6 = copy.copy(Y_test['e_6_paso'])
ee = copy.copy(Y_test['e_trajec'])
# Eliminamos los puntos los nan
e1.dropna(inplace=True)
e6.dropna(inplace=True)
ee.dropna(inplace=True)
# Calculamos el resultado
RMSE_1_paso = np.sqrt(sum(e1 ** 2) / len(e1))
RMSE_6_paso = np.sqrt(sum(e6 ** 2) / len(e6))
RMSE_trajec = np.sqrt(sum(ee ** 2) / len(ee))
# Agregamos el resultado a un dataframe
resultados.loc['RMSE', '1 paso'] = RMSE_1_paso
resultados.loc['RMSE', '6 paso'] = RMSE_6_paso
resultados.loc['RMSE', 'trajec'] = RMSE_trajec
# Lo desplegamos por consola
print(' - Testing')
print('RMSE 1 paso : {:.3f}'.format(RMSE_1_paso))
print('RMSE 6 pasos: {:.3f}'.format(RMSE_6_paso))
print('RMSE traject: {:.3f}'.format(RMSE_trajec))

In [ ]:
# Parametros
f_ini = pd.Timestamp('2020-01-29 00:00:00')
f_fin = pd.Timestamp('2020-01-29 23:59:59')

# Graficamos
fig, ax = plt.subplots()

ax.plot(Y_test['y+6'][f_ini: f_fin], color='lightcoral', 
        linestyle='--', label='$\hat{y}_{t + 6}$')
ax.plot(Y_test['y+5'][f_ini: f_fin], color='indianred', 
        linestyle='--', label='$\hat{y}_{t + 5}$')
ax.plot(Y_test['y+4'][f_ini: f_fin], color='brown', 
        linestyle='--', label='$\hat{y}_{t + 4}$')
ax.plot(Y_test['y+3'][f_ini: f_fin], color='firebrick', 
        linestyle='--', label='$\hat{y}_{t + 3}$')
ax.plot(Y_test['y+2'][f_ini: f_fin], color='maroon', 
        linestyle='--', label='$\hat{y}_{t + 2}$')
ax.plot(Y_test['y+1'][f_ini: f_fin], color='darkred', 
        linestyle='--', label='$\hat{y}_{t + 1}$')
ax.plot(Y_test['y'][f_ini: f_fin], color='C0', linewidth=3.0
       , label='$y_{t}$')

# Parametros
ax.grid(True)
ax.set_ylim([0, 450])
ax.set_xlim([f_ini, f_fin])
ax.set_ylabel('Glucose [mg/dL]')
ax.set_xlabel('Time')

# leyenda
handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
#labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles, labels, loc= 'upper center', ncol=4, fancybox=True, shadow=True)

# Formato de las fechas
date_form = mdates.DateFormatter('%H:%M')
ax.xaxis.set_major_formatter(date_form)

y_size = 4.2
x_size = 3 * y_size
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/grafico_tiempo_na_30_1'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

In [ ]:
# Parametros
f_ini = pd.Timestamp('2020-01-29 00:00:00')
f_fin = pd.Timestamp('2020-01-29 23:59:59')

# Graficamos
fig, ax = plt.subplots()

ax.plot(Y_test['y+6'][f_ini: f_fin], color='lightcoral', 
        linestyle='--', label='$\hat{y}_{t + 6}$')
ax.plot(Y_test['y'][f_ini: f_fin], color='C0', linewidth=3.0
       , label='$y_{t}$')

# Parametros
ax.grid(True)
ax.set_ylim([0, 450])
ax.set_xlim([f_ini, f_fin])
ax.set_ylabel('Glucosa [mg/dL]')
ax.set_xlabel('Tiempo')

# leyenda
handles, labels = ax.get_legend_handles_labels()
handles.reverse()
labels.reverse()
#labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles, labels, loc= 'upper center', ncol=4, fancybox=True, shadow=True)

# Formato de las fechas
date_form = mdates.DateFormatter('%H:%M')
ax.xaxis.set_major_formatter(date_form)

y_size = 4.2
x_size = 3 * y_size
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/grafico_tiempo_na_2_30'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

### Gráfico del error

* Grafico del error del training

In [ ]:
# Parametros
f_ini = y_train.index[0]
f_fin = y_train.index[-1]

# Graficamos
fig, ax = plt.subplots()

ax.plot(Y_train['e_1_paso'][f_ini: f_fin], color='C0', linestyle='-', label='$|e_{t + 1}|$')
ax.plot(Y_train['e_6_paso'][f_ini: f_fin], color='C1', linestyle='-', label='$|e_{t + 6}|$')
ax.plot(Y_train['e_trajec'][f_ini: f_fin], color='C2', linestyle='-', label='$e_{trajectory}$')

# Parametros
ax.grid(True)
ax.set_ylim([-150, 150])
ax.set_xlim([f_ini, f_fin])
ax.set_ylabel('Glucose [mg/dL]')
ax.set_xlabel('Time')

# leyenda
ax.legend(ncol=4, fancybox=True, shadow=True)

# Formato de las fechas
date_form = mdates.DateFormatter('%d-%m %H:%M')
ax.xaxis.set_major_formatter(date_form)

y_size = 4.2
x_size = 3 * y_size
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/error_grafico_training_n_a_30'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

* Gráfico del error del testing

In [ ]:
# Parametros
f_ini = y_test.index[0]
f_fin = y_test.index[-1]

# Graficamos
fig, ax = plt.subplots()

ax.plot(Y_test['e_1_paso'][f_ini: f_fin], color='C0', linestyle='-', label='$|e_{t + 1}|$')
ax.plot(Y_test['e_6_paso'][f_ini: f_fin], color='C1', linestyle='-', label='$|e_{t + 6}|$')
ax.plot(Y_test['e_trajec'][f_ini: f_fin], color='C2', linestyle='-', label='$e_{trajectory}$')

# Parametros
ax.grid(True)
ax.set_ylim([-150, 150])
ax.set_xlim([f_ini, f_fin])
ax.set_ylabel('Glucose [mg/dL]')
ax.set_xlabel('Time')

# leyenda
ax.legend(ncol=4, fancybox=True, shadow=True)

# Formato de las fechas
date_form = mdates.DateFormatter('%d-%m %H:%M')
ax.xaxis.set_major_formatter(date_form)

y_size = 4.2
x_size = 3 * y_size
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/error_grafico_testing_na_30'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

### Análisis en frecuencia del error

* Periodograma para el training

In [ ]:
e1 = Y_train['e_1_paso'].dropna()
N = len(e1)
freq = fftfreq(N, 5*60)
E1 = fft(e1, norm='ortho')
E1_N = abs(E1) ** 2
E1_N = pd.Series(E1_N, index=freq)
E1_N = E1_N[freq > 0]

e6 = Y_train['e_6_paso'].dropna()
N = len(e6)
freq = fftfreq(N, 5*60)
E6 = fft(e6, norm='ortho')
E6_N = abs(E6) ** 2
E6_N = pd.Series(E6_N, index=freq)
E6_N = E6_N[freq > 0]

ee = Y_train['e_trajec'].dropna()
N = len(ee)
freq = fftfreq(N, 5*60)
EE = fft(ee, norm='ortho')
EE_N = abs(EE) ** 2
EE_N = pd.Series(EE_N, index=freq)
EE_N = EE_N[freq > 0]

In [ ]:
# Creamos la figura y el axis
fig, (ax1, ax2) = plt.subplots(1, 2)

# Realizamos el grafico
ax1.loglog(E1_N, color='C0', label=r'$e_{t+1}$')
ax1.loglog(E6_N, color='C1', label=r'$e_{t+6}$')
ax1.loglog(EE_N, color='C2', label=r'$e_{trajectory}$')

ax2.semilogy(E1_N, color='C0', label=r'$e_{t+1}$')
ax2.semilogy(E6_N, color='C1', label=r'$e_{t+6}$')
ax2.semilogy(EE_N, color='C2', label=r'$e_{trajectory}$')

# Configuramos los parametros
ax1.grid(True, which='both')
ax1.set_ylim([10 ** (-2), 10 ** 5])
ax1.legend(fancybox=True, shadow=True)

ax2.grid(True, which='both')
ax2.set_ylim([10 ** (-2), 10 ** 5])
ax2.legend(fancybox=True, shadow=True)

ax1.set_ylabel(r'$\frac{Glucose^2}{Hz}$ [mg$^2$/dL$^2$/Hz]')
ax1.set_xlabel(r'Frequency [Hz]')
ax2.set_xlabel(r'Frequency [Hz]')

x_size = 3 * 4.2
y_size = 1 * x_size / 3
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/error_periodograma_training_na_30'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

* Periodograma para el testing

In [ ]:
e1 = Y_test['e_1_paso'].dropna()
N = len(e1)
freq = fftfreq(N, 5*60)
E1 = fft(e1, norm='ortho')
E1_N = abs(E1) ** 2
E1_N = pd.Series(E1_N, index=freq)
E1_N = E1_N[freq > 0]

e6 = Y_test['e_6_paso'].dropna()
N = len(e6)
freq = fftfreq(N, 5*60)
E6 = fft(e6, norm='ortho')
E6_N = abs(E6) ** 2
E6_N = pd.Series(E6_N, index=freq)
E6_N = E6_N[freq > 0]

ee = Y_test['e_trajec'].dropna()
N = len(ee)
freq = fftfreq(N, 5*60)
EE = fft(ee, norm='ortho')
EE_N = abs(EE) ** 2
EE_N = pd.Series(EE_N, index=freq)
EE_N = EE_N[freq > 0]

In [ ]:
# Creamos la figura y el axis
fig, (ax1, ax2) = plt.subplots(1, 2)

# Realizamos el grafico
ax1.loglog(E1_N, color='C0', label=r'$e_{t+1}$')
ax1.loglog(E6_N, color='C1', label=r'$e_{t+6}$')
ax1.loglog(EE_N, color='C2', label=r'$e_{trajectory}$')

ax2.semilogy(E1_N, color='C0', label=r'$e_{t+1}$')
ax2.semilogy(E6_N, color='C1', label=r'$e_{t+6}$')
ax2.semilogy(EE_N, color='C2', label=r'$e_{trajectory}$')

# Configuramos los parametros
ax1.grid(True, which='both')
ax1.set_ylim([10 ** (-2), 10 ** 5])
ax1.legend(fancybox=True, shadow=True)

ax2.grid(True, which='both')
ax2.set_ylim([10 ** (-2), 10 ** 5])
ax2.legend(fancybox=True, shadow=True)

ax1.set_ylabel(r'$\frac{Glucose^2}{Hz}$ [mg$^2$/dL$^2$/Hz]')
ax1.set_xlabel(r'Frequency [Hz]')
ax2.set_xlabel(r'Frequency [Hz]')

x_size = 3 * 4.2
y_size = 1 * x_size / 3
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/error_periodograma_testing_na_30'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

* Espectro para el training

In [ ]:
e1 = Y_train['e_1_paso'].dropna()
e6 = Y_train['e_6_paso'].dropna()
ee = Y_train['e_trajec'].dropna()

R_e1 = np.correlate(e1, e1, mode='full') / N
R_e6 = np.correlate(e6, e6, mode='full') / N
R_ee = np.correlate(ee, ee, mode='full') / N

N1 = len(R_e1)
N6 = len(R_e6)
Ne = len(R_ee)

# Gamma = N/2
gamma = round(N1 / 2) - 1
phi_E1 = phi_X(R_e1, gamma)

gamma = round(N6 / 2) - 1
phi_E6 = phi_X(R_e6, gamma)

gamma = round(Ne / 2) - 1
phi_EE = phi_X(R_ee, gamma)

In [ ]:
# Creamos la figura y el axis
fig, (ax1, ax2) = plt.subplots(1, 2)

# Realizamos el grafico
ax1.loglog(abs(phi_E1), color='C0', label=r'$e_{t+1}$')
ax1.loglog(abs(phi_E6), color='C1', label=r'$e_{t+6}$')
ax1.loglog(abs(phi_EE), color='C2', label=r'$e_{trajectory}$')

ax2.semilogy(abs(phi_E1), color='C0', label=r'$e_{t+1}$')
ax2.semilogy(abs(phi_E6), color='C1', label=r'$e_{t+6}$')
ax2.semilogy(abs(phi_EE), color='C2', label=r'$e_{trajectory}$')

# Configuramos los parametros
ax1.grid(True, which='both')
#ax1.set_ylim([10 ** (-2), 10 ** 6])
#ax1.set_xlim([10 ** (-6), max(Y_N.index)])
ax1.legend(fancybox=True, shadow=True)

ax2.grid(True, which='both')
#ax2.set_ylim([10 ** (-2), 10 ** 6])
#ax2.set_xlim([10 ** (-6), max(Y_N.index)])
ax2.legend(fancybox=True, shadow=True)

ax1.set_ylabel(r'$\frac{Glucose^2}{Hz}$ [mg$^2$/dL$^2$/Hz]')
ax1.set_xlabel(r'Frequency [Hz]')
ax2.set_xlabel(r'Frequency [Hz]')

x_size = 3 * 4.2
y_size = 1 * x_size / 3
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/error_espectro_training_na_30'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')

* Espectro para el testing

In [ ]:
e1 = Y_test['e_1_paso'].dropna()
e6 = Y_test['e_6_paso'].dropna()
ee = Y_test['e_trajec'].dropna()

R_e1 = np.correlate(e1, e1, mode='full') / N
R_e6 = np.correlate(e6, e6, mode='full') / N
R_ee = np.correlate(ee, ee, mode='full') / N

N1 = len(R_e1)
N6 = len(R_e6)
Ne = len(R_ee)

# Gamma = N/2
gamma = round(N1 / 2) - 1
phi_E1 = phi_X(R_e1, gamma)

gamma = round(N6 / 2) - 1
phi_E6 = phi_X(R_e6, gamma)

gamma = round(Ne / 2) - 1
phi_EE = phi_X(R_ee, gamma)

In [ ]:
# Creamos la figura y el axis
fig, (ax1, ax2) = plt.subplots(1, 2)

# Realizamos el grafico
ax1.loglog(abs(phi_E1), color='C0', label=r'$e_{t+1}$')
ax1.loglog(abs(phi_E6), color='C1', label=r'$e_{t+6}$')
ax1.loglog(abs(phi_EE), color='C2', label=r'$e_{trajectory}$')

ax2.semilogy(abs(phi_E1), color='C0', label=r'$e_{t+1}$')
ax2.semilogy(abs(phi_E6), color='C1', label=r'$e_{t+6}$')
ax2.semilogy(abs(phi_EE), color='C2', label=r'$e_{trajectory}$')

# Configuramos los parametros
ax1.grid(True, which='both')
#ax1.set_ylim([10 ** (-2), 10 ** 6])
#ax1.set_xlim([10 ** (-6), max(Y_N.index)])
ax1.legend(fancybox=True, shadow=True)

ax2.grid(True, which='both')
#ax2.set_ylim([10 ** (-2), 10 ** 6])
#ax2.set_xlim([10 ** (-6), max(Y_N.index)])
ax2.legend(fancybox=True, shadow=True)

ax1.set_ylabel(r'$\frac{Glucose^2}{Hz}$ [mg$^2$/dL$^2$/Hz]')
ax1.set_xlabel(r'Frequency [Hz]')
ax2.set_xlabel(r'Frequency [Hz]')

x_size = 3 * 4.2
y_size = 1 * x_size / 3
fig.set_size_inches(x_size, y_size)
plt.tight_layout()

format_name = 'figs/error_espectro_testing_na_30'
fig.savefig(format_name + '.svg')
fig.savefig(format_name + '.pdf')